In [1]:
pip install qiskit qiskit-machine-learning pandas numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 9.6 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512554 sha256=cb942593193828a52c4810e0669ebb1efb7e5e53bbadd404882e1d81345adffe
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25

In [2]:
pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 59.9 MB/s eta 0:00:00


In [3]:
# Importing libaries

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.primitives import StatevectorEstimator
from qiskit.primitives import Sampler
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.neural_networks import EstimatorQNN
from scipy.optimize import minimize
import time
from sklearn.decomposition import PCA


In [4]:
from scipy.optimize import minimize
from sklearn.metrics import accuracy_score
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN

In [5]:
# Load and preprocess data first
column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label'
]

# Load data
train_data = pd.read_csv('KDDTrain+.txt', names=column_names)
test_data = pd.read_csv('KDDTest+.txt', names=column_names)

# Select numeric columns for initial analysis
numeric_columns = [
    'duration', 'src_bytes', 'dst_bytes', 'count', 'srv_count',
    'dst_host_srv_count', 'serror_rate', 'srv_serror_rate'
]
selected_columns = numeric_columns + ['label']

# Select the columns
train_data = train_data[selected_columns]
test_data = test_data[selected_columns]

# Convert labels to binary
train_data.loc[:, 'label'] = train_data['label'].apply(lambda x: 1 if x != 'normal' else 0)
test_data.loc[:, 'label'] = test_data['label'].apply(lambda x: 1 if x != 'normal' else 0)

# Convert any string numbers to float
for col in numeric_columns:
    train_data[col] = pd.to_numeric(train_data[col], errors='coerce')
    test_data[col] = pd.to_numeric(test_data[col], errors='coerce')

# Fill any NaN values
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

# Define the data preprocessing function
def preprocess_data(X_train, X_test, n_samples=200, n_components=4):
    # Scale data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Reduce dimensions
    pca = PCA(n_components=n_components)
    X_train_reduced = pca.fit_transform(X_train_scaled[:n_samples])
    X_test_reduced = pca.transform(X_test_scaled)

    return X_train_reduced, X_test_reduced


In [6]:
# Enhanced quantum circuit
def create_enhanced_circuit(n_qubits):
    circuit = QuantumCircuit(n_qubits)
    # First rotation layer
    for i in range(n_qubits):
        circuit.h(i)
        circuit.ry(Parameter(f'theta_{i}'), i)

    # Entanglement layer
    for i in range(n_qubits-1):
        circuit.cx(i, i+1)

    # Second rotation layer
    for i in range(n_qubits):
        circuit.ry(Parameter(f'phi_{i}'), i)

    return circuit

In [7]:
# Define the QML Trainer class
class QMLTrainer:
    def __init__(self, n_qubits, X_train, y_train):
        self.feature_map = create_enhanced_circuit(n_qubits)
        self.estimator = Estimator()
        self.qnn = EstimatorQNN(
            circuit=self.feature_map,
            input_params=self.feature_map.parameters[:n_qubits],
            weight_params=self.feature_map.parameters[n_qubits:],
            estimator=self.estimator
        )
        self.X_train = X_train
        self.y_train = y_train
        self.best_loss = float('inf')
        self.best_weights = None

    def loss_fn(self, weights):
        try:
            y_pred = self.qnn.forward(self.X_train, weights)
            loss = -accuracy_score(self.y_train, np.sign(y_pred))

            # Update best weights if better loss
            if loss < self.best_loss:
                self.best_loss = loss
                self.best_weights = weights.copy()

            print(f"Loss: {loss:.4f}")
            return loss
        except Exception as e:
            print(f"Error in loss calculation: {e}")
            return float('inf')


In [8]:
def create_enhanced_circuit(n_qubits):
    circuit = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        circuit.h(i)
        circuit.ry(Parameter(f'theta_{i}'), i)
    for i in range(n_qubits-1):
        circuit.cx(i, i+1)
    for i in range(n_qubits):
        circuit.ry(Parameter(f'phi_{i}'), i)
    return circuit

# Let's create a test class with just the basic structure just in case
class TestQMLTrainer:
    def __init__(self, n_qubits, X_train, y_train):
        print("Initializing TestQMLTrainer")
        self.X_train = X_train
        self.y_train = y_train
        self.feature_map = create_enhanced_circuit(n_qubits)
        self.estimator = Estimator()
        self.qnn = EstimatorQNN(
            circuit=self.feature_map,
            input_params=self.feature_map.parameters[:n_qubits],
            weight_params=self.feature_map.parameters[n_qubits:],
            estimator=self.estimator
        )
        self.best_loss = float('inf')
        self.best_weights = None

    def test_method(self):
        print("Test method works!")

    def loss_fn(self, weights):
        print("Computing loss")
        try:
            y_pred = self.qnn.forward(self.X_train, weights)
            loss = -accuracy_score(self.y_train, np.sign(y_pred))
            if loss < self.best_loss:
                self.best_loss = loss
                self.best_weights = weights.copy()
            return loss
        except Exception as e:
            print(f"Error in loss calculation: {e}")
            return float('inf')

    def train(self, initial_point, maxiter=300):
        print("Training started")
        start_time = time.time()
        opt_result = minimize(
            self.loss_fn,
            initial_point,
            method='COBYLA',
            options={'maxiter': maxiter, 'tol': 1e-3}
        )
        training_time = time.time() - start_time
        print(f"Training completed in {training_time:.2f} seconds")
        return opt_result, self.best_weights

In [9]:
def train(self, initial_point, maxiter=300):
        start_time = time.time()

        opt_result = minimize(
            self.loss_fn,
            initial_point,
            method='COBYLA',
            options={'maxiter': maxiter, 'tol': 1e-3,  'rhobeg': 1.0}
        )

        training_time = time.time() - start_time
        print(f"\nTraining completed in {training_time:.2f} seconds")
        return opt_result, self.best_weights


In [10]:
# Prepare data for quantum model
X_train_data = train_data[numeric_columns].values
X_test_data = test_data[numeric_columns].values
y_train = train_data['label'].values
y_test = test_data['label'].values

In [11]:
# Preprocess data
X_train_reduced, X_test_reduced = preprocess_data(X_train_data, X_test_data, n_samples=200, n_components=4)

print("Data shapes after preprocessing:")
print(f"X_train_reduced shape: {X_train_reduced.shape}")
print(f"X_test_reduced shape: {X_test_reduced.shape}")


Data shapes after preprocessing:
X_train_reduced shape: (200, 4)
X_test_reduced shape: (22544, 4)


In [12]:
# Initialize and train model
n_qubits = X_train_reduced.shape[1]  # Number of features after PCA
trainer = QMLTrainer(n_qubits, X_train_reduced, y_train[:200])
initial_point = np.random.rand(len(trainer.qnn.weight_params))
print("\nStarting model training...")


Starting model training...


<ipython-input-7-072dd654b5a7>:5: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  self.estimator = Estimator()


In [13]:
# Test class creation
trainer = TestQMLTrainer(n_qubits, X_train_reduced, y_train[:200])
print("Trainer created successfully")

Initializing TestQMLTrainer
Trainer created successfully


<ipython-input-8-bb49ee3e7c16>:19: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  self.estimator = Estimator()


In [14]:
# Cell 2: Test basic method
trainer.test_method()

Test method works!


In [15]:
# Cell 3: Test loss function with random weights
initial_point = np.random.rand(len(trainer.qnn.weight_params))
test_loss = trainer.loss_fn(initial_point)
print(f"Test loss: {test_loss}")

Computing loss
Test loss: -0.26


In [16]:
# optimize results
opt_result, best_weights = trainer.train(initial_point, maxiter=300)

Training started
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Computing loss
Training completed in 13.14 seconds


In [17]:
# Evaluate model
print("\nEvaluating model...")
y_pred_test = trainer.qnn.forward(X_test_reduced, best_weights)
accuracy = accuracy_score(y_test, np.sign(y_pred_test))
print(f"\nFinal Test Accuracy: {accuracy * 100:.2f}%")


Evaluating model...

Final Test Accuracy: 91.34%


In [18]:
# Print optimization results
print("\nOptimization Results:")
print(f"Success: {opt_result.success}")

# Safely print number of iterations if available
if hasattr(opt_result, 'nit'):
    print(f"Number of iterations: {opt_result.nit}")
else:
    print("Number of iterations: Not available")

# Print final loss value
if hasattr(opt_result, 'fun'):
    print(f"Final loss value: {opt_result.fun}")
else:
    print(f"Final loss value: {trainer.best_loss}")

# Print additional diagnostic information
print("\nAdditional Information:")
print(f"Best loss found: {trainer.best_loss}")
if trainer.best_weights is not None:
    print(f"Best weights shape: {trainer.best_weights.shape}")
print(f"Optimization message: {opt_result.message}")


Optimization Results:
Success: True
Number of iterations: Not available
Final loss value: -0.955

Additional Information:
Best loss found: -0.955
Best weights shape: (4,)
Optimization message: Optimization terminated successfully.
